<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Product" data-toc-modified-id="Product-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Product</a></span></li><li><span><a href="#User" data-toc-modified-id="User-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>User</a></span><ul class="toc-item"><li><span><a href="#created_at" data-toc-modified-id="created_at-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>created_at</a></span></li><li><span><a href="#updated_at" data-toc-modified-id="updated_at-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>updated_at</a></span></li></ul></li></ul></div>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display
from tqdm import tqdm

import datetime

In [2]:
items_df = pd.read_csv('../data/olist_order_items_dataset.csv')
payments_df = pd.read_csv('../data/olist_order_payments_dataset.csv')
orders_df = pd.read_csv('../data/olist_orders_dataset.csv')
customers_df = pd.read_csv('../data/olist_customers_dataset.csv')
products_df = pd.read_csv('../data/olist_products_dataset.csv')

translation = pd.read_csv('../data/product_category_name_translation.csv')
translation = translation.set_index('product_category_name')['product_category_name_english'].to_dict()

## Product

In [7]:
products = pd.read_csv('../data/prepared/products.csv')
products

,id,product_category,name,created_at,updated_at,base_price,competitor_price
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery,NaN,2018-04-24 16:16:53,2018-04-24 16:16:53,33313.9032,NaN
1,3aa071139cb16b67ca9e5dea641aaa2f,art,NaN,2018-01-31 18:55:36,2018-01-31 18:55:36,757272.9600,NaN
2,96bd76ec8810374ed1b65e291975717f,sports_leisure,NaN,2018-07-09 21:18:01,2018-07-09 21:18:01,243670.8960,NaN
3,cef67bcfe19066a932b7673e239eb23d,baby,NaN,2018-08-03 08:55:50,2018-08-03 08:55:50,342910.2960,NaN
4,9dc1a7de274444849c219cff195d0b71,housewares,NaN,2018-04-11 01:06:37,2018-04-11 01:06:37,115728.4080,NaN
...,...,...,...,...,...,...,...
32946,a0b7d5a992ccda646f2d34e418fff5a0,furniture_decor,NaN,2017-12-29 19:03:07,2018-08-20 17:06:21,213441.0480,NaN
32947,bf4538d88321d0fd4412a93c974510e6,construction_tools_lights,NaN,2018-04-12 11:29:56,2018-08-06 17:28:26,732844.8000,NaN
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,bed_bath_table,NaN,2017-08-21 15:37:43,2018-06-24 13:59:43,389323.8000,NaN
32949,83808703fc0706a22e264b9d75f04a2e,computers_accessories,NaN,2017-10-19 09:26:47,2018-03-06 11:51:44,91300.2480,NaN


## User

In [4]:
customers = customers_df[['customer_id']]
customers['name'] = None
customers = customers.rename(columns={'customer_id': 'id'})
customers = customers.set_index('id')

/tmp/ipykernel_2408/2051282367.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customers['name'] = None


### created_at

In [5]:
purchase_df = customers_df.merge(orders_df[['customer_id', 'order_purchase_timestamp']], on='customer_id')
purchase_df['order_purchase_timestamp'] = pd.to_datetime(purchase_df['order_purchase_timestamp'])
createdAt = purchase_df.groupby('customer_id').order_purchase_timestamp.min().to_frame('created_at')
createdAt.index.name = 'id'

customers = customers.merge(createdAt, on='id')

In [41]:
customers

,name,created_at
id,,
06b8999e2fba1a1fbc88172c00ba8bc7,None,2017-05-16 15:05:35
18955e83d337fd6b2def6b18a428ac77,None,2018-01-12 20:48:24
4e7b3e00288586ebd08712fdd0374a03,None,2018-05-19 16:07:45
b2b6027bc5c5109e529d4dc6358b12c3,None,2018-03-13 16:06:38
4f2d8ab171c80ec8364f7c12e35b23ad,None,2018-07-29 09:51:30
...,...,...
17ddf5dd5d51696bb3d7c6291687be6f,None,2018-04-07 15:48:17
e7b71a9017aa05c9a7fd292d714858e8,None,2018-04-04 08:20:22
5e28dfe12db7fb50a4b2f691faecea5e,None,2018-04-08 20:11:50


### updated_at

In [42]:
purchase_df = customers_df.merge(orders_df[['customer_id', 'order_purchase_timestamp']], on='customer_id')
purchase_df['order_purchase_timestamp'] = pd.to_datetime(purchase_df['order_purchase_timestamp'])
updatedAt = purchase_df.groupby('customer_id').order_purchase_timestamp.max().to_frame('updated_at')
updatedAt.index.name = 'id'
customers = customers.merge(updatedAt, on='id')

In [44]:
# customers.to_csv('../data/prepared/customers.csv')
# data['basePrice'] = data['basePrice'] * 3053.52
customers.read('../data/prepared/customers.csv')
customers

,name,created_at,updated_at
id,,,
06b8999e2fba1a1fbc88172c00ba8bc7,None,2017-05-16 15:05:35,2017-05-16 15:05:35
18955e83d337fd6b2def6b18a428ac77,None,2018-01-12 20:48:24,2018-01-12 20:48:24
4e7b3e00288586ebd08712fdd0374a03,None,2018-05-19 16:07:45,2018-05-19 16:07:45
b2b6027bc5c5109e529d4dc6358b12c3,None,2018-03-13 16:06:38,2018-03-13 16:06:38
4f2d8ab171c80ec8364f7c12e35b23ad,None,2018-07-29 09:51:30,2018-07-29 09:51:30
...,...,...,...
17ddf5dd5d51696bb3d7c6291687be6f,None,2018-04-07 15:48:17,2018-04-07 15:48:17
e7b71a9017aa05c9a7fd292d714858e8,None,2018-04-04 08:20:22,2018-04-04 08:20:22
5e28dfe12db7fb50a4b2f691faecea5e,None,2018-04-08 20:11:50,2018-04-08 20:11:50


In [29]:
import pandas as pd
data = pd.read_csv('../data/prepared/interpolated_demand_history_by_category.csv')
pd.set_option('display.max_rows', 1000)
data = data[data.product_category_name=='informatica_acessorios'].sort_values('order_purchase_timestamp')[['sales', 'price']]

In [30]:
data

,product_category_name,order_purchase_timestamp,sales,price
48,informatica_acessorios,2016-10-04,2,358788.600000
63,informatica_acessorios,2016-10-05,5,231383.531520
92,informatica_acessorios,2016-10-06,10,168487.126560
106,informatica_acessorios,2016-10-07,10,233817.186960
135,informatica_acessorios,2016-10-08,13,231463.862585
164,informatica_acessorios,2016-10-09,12,220242.763800
171,informatica_acessorios,2016-10-10,12,216171.403800
211,informatica_acessorios,2016-10-11,11,190298.142327
229,informatica_acessorios,2016-10-12,11,190298.142327
244,informatica_acessorios,2016-10-13,11,190298.142327


In [27]:
human_predictions = []
for i in range(len(data)-30):
    display(data.iloc[i: i+30])
    human_predictions.append(int(input()))

,sales,price
48,2,358788.600000
63,5,231383.531520
92,10,168487.126560
106,10,233817.186960
135,13,231463.862585
164,12,220242.763800
171,12,216171.403800
211,11,190298.142327
229,11,190298.142327
244,11,190298.142327


11


,sales,price
63,5,231383.531520
92,10,168487.126560
106,10,233817.186960
135,13,231463.862585
164,12,220242.763800
171,12,216171.403800
211,11,190298.142327
229,11,190298.142327
244,11,190298.142327
288,11,190298.142327


11


,sales,price
92,10,168487.126560
106,10,233817.186960
135,13,231463.862585
164,12,220242.763800
171,12,216171.403800
211,11,190298.142327
229,11,190298.142327
244,11,190298.142327
288,11,190298.142327
298,11,190298.142327


...

11


,sales,price
751,11,190298.142327
792,11,190298.142327
807,11,190298.142327
841,11,190298.142327
842,11,190298.142327
885,11,190298.142327
902,11,190298.142327
915,11,190298.142327
938,11,190298.142327
971,11,190298.142327


11


,sales,price
792,11,190298.142327
807,11,190298.142327
841,11,190298.142327
842,11,190298.142327
885,11,190298.142327
902,11,190298.142327
915,11,190298.142327
938,11,190298.142327
971,11,190298.142327
988,11,190298.142327


KeyboardInterrupt: Interrupted by user